In [1]:
import json
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
import torch
import torchvision.transforms as T
import os
import re

from tifffile import tifffile 
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image

In [ ]:
# to delete - Illustration of some samples coming from the dataset
json_file_path = 'Data/large_rock_dataset.json'

# Load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)
print('General information about the data:', data['info'])
dataset =data['dataset']
print('Number of samples  :', len(dataset) )
sample_info = dataset[10]
print('Looking at the the first images:', sample_info ['file_name'])
print('Looking at rocks annotations for the first images:\n', sample_info ['rocks_annotations'][0])

In [ ]:
# to delete - vis
unique_splits = set(sample['split'] for sample in dataset)
print("Unique values in 'split':", unique_splits)

In [10]:
# to delete - Dataset organisation 

# TRAIN/TEST
base_dir = 'dataset_surface_hillshade'
# base_dir = 'dataset_swissimage'
train_images_folder = os.path.join(base_dir, 'train_images')
test_dir = os.path.join(base_dir, 'test')

# Create directories if they don't exist
# os.makedirs(base_dir, exist_ok=True)
# os.makedirs(train_images_folder, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

## DONE:

# Iterate through all samples
# for sample in dataset:
#     file_name = 'Data/swissImage_50cm_patches/' + sample['file_name']
#     split = sample['split']  # Assuming the "split" key indicates train/test/val

#     # Define source and destination paths
#     src_path = file_name  # Assuming file_name contains the full or relative path
#     if split == 'train':
#         dest_dir = train_images_folder
#     elif split == 'test':
#         dest_dir = test_dir
#     # elif split == 'val':
#     #     dest_dir = val_dir
#     else:
#         print(f"Unknown split '{split}' for file '{file_name}'. Skipping.")
#         continue

#     dest_path = os.path.join(dest_dir, os.path.basename(file_name))

#     # Copy file to the appropriate directory
#     try:
#         shutil.copy(src_path, dest_path)
#         print(f"Copied '{file_name}' to '{dest_dir}'")
#     except Exception as e:
#         print(f"Error copying '{file_name}': {e}")

# print("Dataset split completed.")


In [11]:
# to delete - Dataset organisation 

# train_labels
train_labels_folder = os.path.join(base_dir, 'train_labels')

# Create the train_labels directory if it doesn't exist
# os.makedirs(train_labels_folder, exist_ok=True)

# DONE:

# Process images with split == 'train'
# for sample in dataset:
#     if sample['split'] == 'train':
#         # Extract relevant details
#         file_name = sample['file_name']
#         annotations = sample.get('rocks_annotations', [])
        
#         # Create a .txt file for this image
#         base_name = os.path.splitext(os.path.basename(file_name))[0]
#         txt_file_path = os.path.join(train_labels_folder, f"{base_name}.txt")
        
#         # Write annotations to the .txt file
#         with open(txt_file_path, 'w') as txt_file:
#             for annotation in annotations:
#                 txt_file.write(f"{annotation}\n")
        
#         print(f"Created annotation file: {txt_file_path}")

# print("All train annotations have been saved to the 'train_labels' folder.")


In [ ]:
# to delete - Dataset organisation

# Create Validation Set - images
file_count = len([file for file in os.listdir(train_images_folder) if os.path.isfile(os.path.join(train_images_folder, file))])
print(f"Number of images in train set: {file_count}")

# Define folder
val_images_folder = os.path.join(base_dir, 'val_images')
# os.makedirs(val_images_folder, exist_ok=True)

# DONE:

# List all files in the source folder
# files = [file for file in os.listdir(train_images_folder) if os.path.isfile(os.path.join(train_images_folder, file))]

# # Calculate 10% of the total files
# num_files_to_move = max(1, int(len(files) * 0.1))  # Ensure at least one file is moved

# # Randomly select 10% of the files
# files_to_move = random.sample(files, num_files_to_move)

# # Move the selected files
# for file in files_to_move:
#     src_path = os.path.join(train_images_folder, file)
#     dest_path = os.path.join(val_images_folder, file)
#     shutil.move(src_path, dest_path)
#     print(f"Moved '{file}' to '{val_images_folder}'")

# print(f"Moved {len(files_to_move)} files to '{val_images_folder}'.")


In [13]:
# to delete - Dataset organisation 

# Create Validation Set - labels
val_labels_folder = os.path.join(base_dir, 'val_labels')
# os.makedirs(val_labels_folder, exist_ok=True)

# DONE:

# List all image files in val_images folder (excluding extensions)
# val_image_files = {os.path.splitext(file)[0] for file in os.listdir(val_images_folder) if os.path.isfile(os.path.join(val_images_folder, file))}

# # Move matching label files from train_labels to val_labels
# for label_file in os.listdir(train_labels_folder):
#     # Get the base name (without extension) of the label file
#     base_name = os.path.splitext(label_file)[0]
    
#     if base_name in val_image_files:
#         src_path = os.path.join(train_labels_folder, label_file)
#         dest_path = os.path.join(val_labels_folder, label_file)
#         shutil.move(src_path, dest_path)
#         print(f"Moved '{label_file}' to '{val_labels_folder}'")

# print("Matching label files moved to 'val_labels' folder.")




In [ ]:
# to delete - tif to jpg - DONE:

# Path to the folder containing .tif images
# folder_path = 'dataset_surface_hillshade/test'

# # Iterate through all files in the folder
# for file in os.listdir(folder_path):
#     if file.endswith('.tif'):
#         # Full path to the .tif file
#         tif_path = os.path.join(folder_path, file)
        
#         # Open the .tif file
#         try:
#             with Image.open(tif_path) as img:
#                 # Define the output path with the same name but .jpg extension
#                 jpg_path = os.path.join(folder_path, file.replace('.tif', '.jpg'))
                
#                 # Convert and save as JPG
#                 img.convert('RGB').save(jpg_path, 'JPEG')
                
#                 # Remove the original .tif file
#                 os.remove(tif_path)
#                 print(f"Converted and replaced: {file} -> {jpg_path}")
        
#         except Exception as e:
#             print(f"Error processing {file}: {e}")

# print("All .tif files have been converted to .jpg and replaced.")

In [14]:
# to delete - label in form for Yolov8 :

# Input and output directories
label_input_folder = val_labels_folder  # Folder containing original label files
label_output_folder = os.path.join(base_dir, 'yolo_val_labels')  # Folder for YOLO-compliant labels
# os.makedirs(label_output_folder, exist_ok=True)

# DONE:

# YOLOv8 assumes constant bbox size based on your description
# bbox_width = 10 / 640  # Normalized width
# bbox_height = 10 / 640  # Normalized height

# # Process each label file
# for label_file in os.listdir(label_input_folder):
#     if label_file.endswith('.txt'):  # Process only text files
#         input_path = os.path.join(label_input_folder, label_file)
#         output_path = os.path.join(label_output_folder, label_file)

#         with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
#             # Read each line in the file and extract dictionaries
#             for line in infile:
#                 line = line.strip()
#                 if not line:
#                     continue
                
#                 # Parse the dictionary using regex
#                 match = re.search(r"'relative_within_patch_location': \[(\d+\.\d+), (\d+\.\d+)\]", line)
#                 if match:
#                     x_center = float(match.group(1))  # Normalized x_center
#                     y_center = float(match.group(2))  # Normalized y_center

#                     # Write to YOLO format: class_id, x_center, y_center, width, height
#                     class_id = 0  # Assuming 'rock' class is class 0
#                     outfile.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")
        
#         print(f"Processed: {label_file}")

# print("Conversion to YOLOv8 format completed.")

In [ ]:
# MAIN - define dataset and visualise the first tensor (normalised img) - DO NOT TOUCH

import dataset as dt

image_folder = "dataset_swissimage/train_images"
label_folder = "dataset_swissimage/yolo_train_labels"

mean, std = dt.calculate_mean_std(image_folder)

# Create Dataset
dataset = dt.RockDetectionDataset(image_folder, label_folder, mean, std, augment=False)

# Iterate through dataset
for idx, (aug_images, aug_labels) in enumerate(dataset):
    image_name = dataset.image_files[idx]  # Get the name of the current image
    print(f"Image Name: {image_name}")
    print(f"Image Shape: {aug_images.size()}")
    print(f"Labels: {aug_labels}")
    break

In [ ]:
# MAIN - Augmentation and save augmented data - DO NOT TOUCH

# Parameters
batch_size = 16  # Adjust based on your system's memory
base_dir = 'dataset_surface_hillshade'
output_image_folder = os.path.join(base_dir, 'augmented_train_images')
output_label_folder = os.path.join(base_dir, 'augmented_train_labels')
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

# Batch processing loop
batch_images, batch_labels, batch_names = [], [], []

for idx in range(len(dataset)):
    # Load original image and labels
    original_image, original_labels = dataset[idx]
    original_image_name = dataset.image_files[idx]  # Get image name

    # Convert tensor to PIL image if needed
    if isinstance(original_image, torch.Tensor):
        original_image = dt.denormalize(original_image.clone(), mean, std)
        original_image = T.ToPILImage()(original_image)

    # Perform augmentations
    augmented_images, augmented_labels = dt.transform_train_with_labels(original_image, original_labels)

    # Append to batch
    batch_images.append(augmented_images)
    batch_labels.append(augmented_labels)
    batch_names.append(original_image_name)

    # When batch is full, save the batch
    if len(batch_images) >= batch_size:
        dt.save_augmented_data_batch(
            batch_images, batch_labels, batch_names,
            output_image_folder, output_label_folder,
            augmentation_names=[f"aug_{i + 1}" for i in range(len(augmented_images))],
            mean=mean, std=std
        )
        # Reset batch
        batch_images, batch_labels, batch_names = [], [], []

# Save remaining images in the last batch
if len(batch_images) > 0:
    dt.save_augmented_data_batch(
        batch_images, batch_labels, batch_names,
        output_image_folder, output_label_folder,
        augmentation_names=[f"aug_{i + 1}" for i in range(len(augmented_images))],
        mean=mean, std=std
    )